In [1]:
from astroquery.utils.tap.core import Tap
import vaex
from datetime import datetime
from time import time
from glob import glob
import sys
from matplotlib import pyplot as plt
from os.path import join, abspath
from os import pardir, curdir, mkdir
import numpy as np
import pandas as pd

In [2]:
# import utils
util_dir = abspath(pardir)
sys.path.insert(0, util_dir)

from utils import timeout, appendName, progressbar

In [3]:
name = "TWOMASS"
root_data_dir = abspath(join(pardir, "Data"))
data_dir = join(root_data_dir, name)
try:
  mkdir(data_dir)
  print(f"Creating {name} dir in Data dir")
except FileExistsError:
  print("Directory already exist. Good to go!")
data_dir

Directory already exist. Good to go!


'/home2/s20321005/Thesis-Project/Data/TWOMASS'

In [4]:
# 2MASS,
tap_tmass = Tap(url="https://irsa.ipac.caltech.edu/TAP/sync")

columns_tmass = ["ra", "dec","j_m", "h_m", "k_m"]
columns_tmass_names = ["ra", "dec", "Jmag", "Hmag", "Kmag"]

In [5]:
# divide into 360 RAs, depend on preference
ras = np.arange(0,361, 10).astype(int)
ras

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360])

In [6]:
des = np.linspace(-90, 90, 10)
des

array([-90., -70., -50., -30., -10.,  10.,  30.,  50.,  70.,  90.])

In [7]:
TOP = 500_000 # cap maximum rows for each response, so that the response is not exploding

# ra0 for lower boundry and ra1 for upper boundary
# same with dec0 and dec1
# download 2MASS with J < 13.5
for i, (ra0, ra1) in enumerate(zip(ras[:-1], ras[1:])):
    if i > 6: break
    df_com = [] #initial table
    time0 = time() 
    print(f"{i} downloading... {ra0}-{ra1}")
    progressbar(0)
    j = 0
    while j < len(des) - 1:
        dec0 = des[j]
        dec1 = des[j+1]
        
        query_tmass = f"""
        SELECT TOP {TOP} {", ".join(columns_tmass)} 
        FROM fp_psc
        WHERE ra BETWEEN {ra0} AND {ra1}
        AND dec BETWEEN {dec0} AND {dec1}
        AND j_m < 13.5
        """
        job_tmass = timeout(tap_tmass.launch_job, args=(query_tmass,), timeout_duration=180)
        if job_tmass == None: 
            print(job_tmass)
            print("fail to fetch tmass")
            print("length = ", len(df_com))
            continue
        result_tmass = job_tmass.get_results()
        progressbar((j+1)/(len(des)-1)*100)
        df_tmass = result_tmass.to_pandas()
        df_tmass.columns = columns_tmass_names
        df_tmass = vaex.from_pandas(df_tmass)
        if len(df_com) == 0:
            df_com = df_tmass
        else:
            df_com = df_com.concat(df_tmass)
        j += 1
        # break
    time1 = time()  
    df_com.export(join(data_dir, f"tmass-{ra0:03d}-{ra1:03d}.hdf5"), progress=True)
    print(f"{len(df_com)} || {round((time1-time0)/60, 2)}m")
    print(f"{i} saved {ra0}-{ra1} || {datetime.now()}")

0 downloading... 0-10
export(hdf5) [########################################] 100.00% elapsed time  :     0.07s =  0.0m =  0.0h
 528108 || 9.7m
0 saved 0-10 || 2022-07-17 02:46:43.014601
1 downloading... 10-20
export(hdf5) [########################################] 100.00% elapsed time  :     0.06s =  0.0m =  0.0h
 507529 || 12.32m
1 saved 10-20 || 2022-07-17 02:59:02.106177
2 downloading... 20-30
export(hdf5) [########################################] 100.00% elapsed time  :     0.06s =  0.0m =  0.0h
 501555 || 10.52m
2 saved 20-30 || 2022-07-17 03:09:33.201714
3 downloading... 30-40
export(hdf5) [########################################] 100.00% elapsed time  :     0.05s =  0.0m =  0.0h
 515716 || 11.48m
3 saved 30-40 || 2022-07-17 03:21:02.164384
4 downloading... 40-50
export(hdf5) [########################################] 100.00% elapsed time  :     0.07s =  0.0m =  0.0h
 524734 || 11.52m
4 saved 40-50 || 2022-07-17 03:32:33.230023
5 downloading... 50-60
export(hdf5) [############

In [11]:
df_com

#,ra,dec,Jmag,Hmag,Kmag
0,240.01014,-83.193924,13.243,12.723,12.664
1,240.000307,-80.888763,13.493,13.099,12.924
2,240.008543,-78.211967,10.554,10.269,10.193
3,240.00093,-89.657791,13.193,12.632,12.495
4,240.001039,-84.839127,10.048,9.91,9.865
...,...,...,...,...,...
"10,645",249.976756,-80.378571,13.402,12.809,12.736
"10,646",249.962133,-84.364761,13.22,12.661,12.603
"10,647",249.971772,-79.018723,12.848,12.519,12.48
"10,648",249.9973,-84.4104,12.269,11.773,11.681
